In [1]:
import numpy as np

In [2]:
import MeCab as mc
import mojimoji
m = mc.Tagger("-Ochasen")

In [3]:
def mecab_tokenizer(text: str):
    """
    テキストを分かち書きするメソッド
    :param text: 分割したいテキスト
    :return: 分割後のテキスト
    """

    node = m.parseToNode(text)
    word_list = list()
    while node:
        if node.surface != "":
            res = node.feature.split(",")
            word_type = res[0]
            if word_type in ['名詞', "動詞", "形容詞", "副詞"]:  # 名詞, 動詞, 形容詞, 副詞のみを抽出
                basic_word = res[6]
                if basic_word != "*":
                    word_list.append(basic_word)
                else:
                    word_list.append('[UNK]')  # 未知語の場合は[UNK]トークンに置き換え
        node = node.next
        if node is None:
            break
    return word_list

In [4]:
def clean_text(text: str):
    """
    テキストの正規化
    :param text: 正規化するテキスト
    :return: 正規化後のテキスト
    """
    text = mojimoji.han_to_zen(text, digit=False, ascii=False)  # 半角文字を全角文字に統一(数字, 英語以外)
    text = mojimoji.zen_to_han(text, kana=False)  # 全角文字を半角文字に統一(かな以外)
    text = text.lower()  # 小文字に統一
    return text

In [5]:
def vectorize_word(text: str):
    """
    元word_analysis()
    ベクトル化を行うメソッド
    :param text: ベクトル化するテキスト
    :return: ベクトル化したテキスト(array型)
    """

    text = clean_text(text)

    V = list()  # 文章のベクトル(200次元)を格納

    # 文章の対して, 文章中の単語のベクトルの平均を求める処理を行う
    word_list = mecab_tokenizer(text)
    v = np.array([0.0] * 200)
    word_num = 0
    for word in word_list:
        try:
            v += np.array(model[word])
        except KeyError as error:
            continue
        except ValueError as verror:
            continue
        word_num += 1

    try:
        v = v / word_num
    except ZeroDivisionError as e:
        print(f'ZeroDivisionError: {e}')

    return v

def cos_sim(v1, v2):
    """
    cos類似度を計算
    :param v1: word2vecによりベクトル化したテキスト1(ndarray型)
    :param v2: word2vecによりベクトル化したテキスト2(ndarray型)
    :return: 二つのテキストのcos類似度
    """
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [6]:
import pickle

In [7]:
with open('word2vec_model.pickle', mode='rb') as f:
    model = pickle.load(f)

In [8]:
with open('E:/jupyter_workplace/研究/KENBUN_jupyter/headline_list.pickle', mode='rb') as f:
    healdline_list = pickle.load(f)

In [9]:
def get_all_similarity(headline_list, speech_text: str):
    """
    全ての見出しと発言の類似度分析を行い, 類似度で降順にソートして返す
    :param headline_list: 見出しをまとめたリスト<br> ひとつの要素に(ファイルパス, 番号, (見出し, ベクトル)) -> の形で格納している
    :param speech_text: 発言
    :return: 類似度で降順にソートしたリスト [((会社名, 出版年, 出版付き, 番号, 見出し), 類似度)] -> この形になっている
    """
    # speech_text = '物価下がらなかったのね'
    speech_v = vectorize_word(speech_text)  # 発言をベクトル化

    similarity_dic = {}

    for company, year, month, day, headline, vector in headline_list:
        cs = cos_sim(speech_v, vector)  # 発言と見出しテキストのcos類似度を計算

        if cs >= 0.75:  # cos類似度が閾値以上であれば提示する見出しリストに加える
            similarity_dic[(company, year, month, day, headline)] = cs

    return sorted(similarity_dic.items(), key=lambda x: x[1], reverse=True)

In [10]:
# test_keyword = '大分'
test_keyword = '''満洲の 宇佐神宮奉贊會
寄附に着手
'''

In [11]:
mecab_tokenizer(test_keyword)

['洲', '宇佐', '神宮', '奉る', '[UNK]', '寄附', '着手']

In [12]:
vectorize_word(test_keyword)

array([ 0.03348364,  0.06445007,  0.05366782,  0.11286375,  0.19991481,
        0.11283206, -0.00546312,  0.01312535, -0.31217946,  0.06201501,
        0.07746638, -0.02019484,  0.09661527, -0.04303557,  0.48072759,
       -0.10658365, -0.12154092, -0.24083348, -0.05821623, -0.17440163,
        0.42715107,  0.03388356, -0.42426558, -0.00771238,  0.17625025,
       -0.20092493, -0.08303688,  0.17103244, -0.28456118,  0.13045597,
        0.07860022, -0.16599423,  0.10534596, -0.02443682,  0.04768699,
       -0.1633281 , -0.25153997,  0.5074919 , -0.39685371,  0.03400588,
       -0.26841549, -0.03718666,  0.01605564, -0.32010584, -0.18536303,
        0.08803011,  0.29316608, -0.19452687,  0.05857726,  0.26638001,
       -0.12674496, -0.13380651,  0.14159116,  0.19015103, -0.28474484,
       -0.01716992, -0.19827748, -0.04255977, -0.19875314,  0.12507856,
        0.12612418, -0.33665423,  0.27133924, -0.1166627 ,  0.33896789,
       -0.03435695,  0.01210771, -0.24524508,  0.3295261 ,  0.33

In [16]:
mecab_tokenizer('大分県には色んな生産物があるな')

['大分', '県', '生産', '物', 'ある']

In [15]:
get_all_similarity(headline_list=healdline_list, speech_text='大分')

[(('大分合同新聞', '1942年', '7月', '23-a-3_18', '2にゃい\nに對策\n大分'), 1.0000000000000002),
 (('大分合同新聞', '1943年', '8月', '26-a-3_11', '什隊 大分發招'), 1.0000000000000002),
 (('大分合同新聞', '1946年', '6月', '27-a-2_01', '「生產大分,頗落'), 1.0000000000000002),
 (('大分合同新聞', '1949年', '11月', '6-a-2_08', '大分縣將棋人會'), 1.0000000000000002),
 (('大分合同新聞', '1949年', '2月', '25-a-2_01', '(サリナ大分) マオマオ'), 1.0000000000000002),
 (('大分合同新聞', '1950年', '3月', '18-a-2_05', '大分縣圍大會附'), 1.0000000000000002),
 (('大分合同新聞', '1950年', '3月', '19-y-2_06', '大分縣回售0'), 1.0000000000000002),
 (('大分合同新聞', '1950年', '3月', '15-y-2_05', '大分縣回售AO'), 1.0000000000000002),
 (('大分合同新聞', '1952年', '1月', '27-y-2_10', '大分别府对抗棋戰'), 1.0000000000000002),
 (('大分合同新聞', '1952年', '2月', '13-y-2_08', '大分别府对抗棋戰'), 1.0000000000000002),
 (('大分合同新聞', '1952年', '2月', '24-y-2-3_12', '大分别府对抗棋戰'), 1.0000000000000002),
 (('大分合同新聞', '1953年', '6月', '8-a-3_13', '大分に'), 1.0000000000000002),
 (('大分合同新聞', '1954年', '5月', '17-a-3_08', '大分\n6A='), 1.0000000000000002),
 (('大分合同新聞', '1956年', '10月'

In [17]:
vectorize_word('大分県には色んな生産物があるな')

array([-0.11528872, -0.15364979,  0.23481922, -0.07247004,  0.25310266,
        0.01120286,  0.10584415,  0.22160915, -0.1655497 , -0.11020138,
        0.07699556, -0.05614914, -0.03005803, -0.1982579 ,  0.04406125,
       -0.22905995,  0.01268337, -0.04207285,  0.08256962, -0.22879145,
        0.12713098,  0.04428437, -0.32544373, -0.01408221, -0.14883337,
       -0.01052284,  0.05911269,  0.08866569,  0.00620107,  0.07817806,
        0.13202267, -0.23060988, -0.1930087 , -0.19375184,  0.0165071 ,
       -0.15476895,  0.10347204,  0.43544834, -0.38747643, -0.11302158,
       -0.14944094, -0.03861373,  0.03184699,  0.02702231, -0.18330479,
       -0.33876312,  0.14085589, -0.19833781,  0.1226432 ,  0.04212445,
       -0.14895012, -0.15936451,  0.02941225,  0.22246278,  0.01926173,
       -0.14478249, -0.17292344,  0.06788123, -0.32460023,  0.10930792,
        0.04482908, -0.41609085,  0.25129029, -0.17748152,  0.09444287,
       -0.07959196,  0.15792554, -0.23096833,  0.1920645 ,  0.11

In [10]:
get_all_similarity(headline_list=healdline_list, speech_text='別府')

[(('大分合同新聞', '1946年', '1月', '18-a-2_10', '觀光別府を'), 1.0000000000000002),
 (('大分合同新聞', '1950年', '2月', '21-a-2_02', '別府に觀光ホテ'), 1.0000000000000002),
 (('大分合同新聞', '1955年', '8月', '26-y-1_07', '別府のリンタク'), 1.0000000000000002),
 (('大分合同新聞', '1956年', '9月', '7-a-3_08', '別府に辻強盗'), 1.0000000000000002),
 (('大分合同新聞', '1965年', '9月', '10-y-2_03', '別府'), 1.0000000000000002),
 (('大分合同新聞', '1967年', '5月', '25-a-4_04', '別府觀光'), 1.0000000000000002),
 (('大分新聞', '1918年', '8月', '31-a-4_05', '別府の'), 1.0000000000000002),
 (('大分新聞', '1919年', '12月', '18-a-6_05', '別府兒の'), 1.0000000000000002),
 (('大分新聞', '1920年', '10月', '22-a-7_06', '別府檢藝妓溫習會 を觀E'), 1.0000000000000002),
 (('大分新聞', '1922年', '10月', '17-a-3_11', 'SLX*\nに関する別府'), 1.0000000000000002),
 (('大分新聞', '1928年', '11月', '27-a-9_15', '別府の'), 1.0000000000000002),
 (('大分新聞', '1932年', '3月', '4-a-6_06', '新別府\n將來大別府の'), 1.0000000000000002),
 (('大分新聞', '1933年', '4月', '18-a-3_13', 'PLE,\n別府演能大會\n| _\n,_='),
  1.0000000000000002),
 (('大分新聞', '1934年', '11月', '25-a-3_10', '

In [11]:
get_all_similarity(headline_list=healdline_list, speech_text='野球')

[(('大分合同新聞', '1960年', '10月', '16-a-8_04', '野球'), 1.0000000000000002),
 (('大分合同新聞', '1961年', '4月', '1-a-5_01', '(スロ野球) (-AM戰)'), 1.0000000000000002),
 (('大分合同新聞', '1963年', '8月', '15-y-3_07', '野球赛'), 1.0000000000000002),
 (('大分合同新聞', '1964年', '2月', '4-a-5_01', 'キプロ野球\n#r176 (414'),
  1.0000000000000002),
 (('大分合同新聞', '1965年', '2月', '16-a-5_02', 'フロ野球'), 1.0000000000000002),
 (('大分合同新聞', '1967年', '3月', '8-a-7_03', '门口野球59'), 1.0000000000000002),
 (('大分新聞', '1929年', '10月', '28-y-1_02', 'スポンチ野球'), 1.0000000000000002),
 (('大分新聞', '1930年', '10月', '27-a-1_03', '野球'), 1.0000000000000002),
 (('大分新聞', '1931年', '11月', '23-a-2_12', '野球大會'), 1.0000000000000002),
 (('大分新聞', '1932年', '10月', '26-y-2_11', '野球大會'), 1.0000000000000002),
 (('大分新聞', '1934年', '5月', '14-a-3_11', '| ||\n||||| |||||||||\n野球'),
  1.0000000000000002),
 (('豊州新報', '1932年', '8月', '4-a-4_15', 'カツプ軟野球'), 1.0000000000000002),
 (('大分合同新聞', '1952年', '10月', '26-a-2_06', '縣下高校野球大會'), 0.8939171608670495),
 (('大分合同新聞', '1952年', '10月', '27-y-